In [ ]:
def replace_tag_with_last_method_name(tags, index):
    """
    Replaces the value at the specified index in the given list of tags with the last method name.

    Args:
        tags (list): The list of tags.
        index (int): The index of the tag to be replaced.

    Returns:
        list: The updated list of tags with the value at the specified index replaced with the last method name.
    """
    for index in range(len(tags.columns)):
        tags[index] = tags[index].str.split('.').str[-1]
    return tags

In [23]:
import pandas as pd
data = pd.read_csv('test.csv')
pd.set_option('display.float_format', lambda x: '%.4f' % x)
# cut digits using lambda function
data = data.round(4)

def extract_tags(data):
    """
    Extracts tags from the given data.

    Parameters:
    data (pandas.DataFrame): The input data containing a 'Tags' column.

    Returns:
    pandas.DataFrame: A DataFrame with the tags extracted from the 'Tags' column.
    """
    return data['Tags'].str.split(',', expand=True).fillna('None')

def replace_tag_with_last_method_name(tags):
    """
    Replaces each tag in the given DataFrame with the last method name in the tag.

    Args:
        tags (DataFrame): The DataFrame containing the tags.

    Returns:
        DataFrame: The DataFrame with tags replaced by the last method name.
    """
    for index in range(len(tags.columns)):
        tags[index] = tags[index].str.split('.').str[-1]
    return tags

def combime_tags_into_new_name(tags):
    """
    Combines the tags in each row into a new name.

    Args:
        tags (DataFrame): The DataFrame containing the tags.

    Returns:
        DataFrame: The DataFrame with the tags combined into a new name.
    """
    # ignore the first column
    new_tags = tags.iloc[:, 1:]
    tags['method'] = new_tags.apply(lambda x: '+'.join(x), axis=1)
    return tags


tags = extract_tags(data)
tags = replace_tag_with_last_method_name(tags)
tags = combime_tags_into_new_name(tags)
data = pd.concat([data, tags], axis=1)
data.rename(columns={'test/cvr_auc': tags[0][0]+'/cvr_auc',
                     'test/ctr_auc': tags[0][0]+'/ctr_auc',
                     'test/ctcvr_auc': tags[0][0]+'/ctcvr_auc'}, inplace=True)
data['Name'] = data['method']
data.rename(columns={'Name': 'Method'}, inplace=True)
new_data = data.iloc[:,:4]
new_data.to_csv('new_data.csv', index=False)

